In [ ]:
# !pip install tensorflow==3.2.0
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm 
import random, time, os
from sklearn.model_selection import train_test_split
import pandas as pd

if not os.path.isdir("/gdrive"):
    from google.colab import drive
    drive.mount("/gdrive")

!test -d data && ls -l data/ || unzip /gdrive/MyDrive/dataset/shopee-code-league-2020-product-detection.zip -d data 1>/dev/null

tf.__version__
# Tutorial
# https://www.kaggle.com/fadheladlansyah/product-detection-effnetb5-aug-tta

total 4604
drwxr-xr-x 4 root root    4096 Feb 19 18:38 resized
-rw-r--r-- 1 root root  487458 Jun 20  2020 test.csv
-rw-r--r-- 1 root root 4215698 Jun 20  2020 train.csv


'2.4.1'

## 1. Input

In [ ]:
def get_data():
    train =  tf.keras.preprocessing.image_dataset_from_directory(
        "data/resized/train/",
        validation_split = .1,
        subset = "training",
        seed = 1,
        labels     = "inferred",
        label_mode = "int",
        image_size = (299, 299)
    )

    validation =  tf.keras.preprocessing.image_dataset_from_directory(
        "data/resized/train/",
        validation_split = .1,
        subset = "training",
        seed = 1,
        labels     = "inferred",
        label_mode = "int",
        image_size = (299, 299)
    )
    return train, validation

## 2. Layers and Models

### 2.1 Preprocessing input

In [ ]:
class Preprocess(tf.keras.layers.Layer):
    def __init__(self):
        super(Preprocess, self).__init__()
    
    def call(self, X):
        X = tf.keras.applications.efficientnet.preprocess_input(X)
        return X

### 2.2. Augmentation layer

In [ ]:
# # Cutmix
# def CutMix(self, X, y, lamda=.66):
#     #N: number of samples within the batch, (W, H) is img size, C is number of img channel (default = 3)
#     if self.N is None:
#         self.N, self.W, self.H, _ = X.shape

#     imgs = []; labels=[]
#     for i in range(self.N):
#         # select image
#         k = np.random.randint(0, self.N)

#         r_w, r_h = (1 - self.lamda)**.5 * self.W, (1 - self.lamda)**.5 * self.H
#         r_w, r_h = int(r_w), int(r_h)
#         r_x, r_y = np.random.randint(0, self.W - r_w), np.random.randint(0, self.H - r_h)

#         # img
#         X_i = X[i,:,:,:]
#         X_k = X[k,:,:,:]

#         y_i = y[i,:]
#         y_k = y[k,:]

#         # Mask
#         M = np.ones_like(X_i)
#         M[r_x:r_x+r_w,r_y:r_y+r_h,:] = 0

#         img = M*X_i + (1-M)*X_k
#         label = y_i * self.lamda + y_k * (1 - self.lamda)

#         imgs.append(img)
#         labels.append(label)

#     return imgs, labels


# class CutOut(tf.keras.layers.Layer):
#     def __init__(self, a):
#         super(CutOut, self).__init__()
#         self.a = a

#     def call(self, X):
#         print(X)
        
#         _, W, H, _ = X.shape

#         w, h = self.a**.5 * W, self.a**.5 * H
#         w, h = int(w), int(h)
#         x, y = np.random.randint(0, W - w), np.random.randint(0, H - h)

#         # Masking
#         M = np.ones_like(X)
#         M[:,x:x+w,y:y+h,:] = 0
        
#         return M*X

In [ ]:
class Augmentation(tf.keras.layers.Layer):
    def __init__(self):
        super(Augmentation, self).__init__()
        # self.cutout = CutOut(.1)
    
    def call(self, X):
        X = tf.image.random_flip_left_right(X)
        X = tf.image.random_brightness(X, max_delta=0.5)
        X = tf.image.random_contrast(X, lower=0.75, upper=1.2)
        # X = self.cutout(X)
        return X

### 2.3 Model

In [ ]:
def get_model():
    with tf.device("/device:GPU:0"):
        # Load efficient net
        base = tf.keras.applications.efficientnet.EfficientNetB5(
            include_top = False,
            weights="imagenet",
            pooling = None
        )
        base.trainable=False

        net = tf.keras.models.Sequential([
            Preprocess(),
            Augmentation(),
            base,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(42, activation="softmax")
        ])

        net.compile(
            optimizer = tf.keras.optimizers.SGD(learning_rate=.1),
            loss = "sparse_categorical_crossentropy",
            metrics = "accuracy"
        )

        return net

## 3. Training loop

### 3.1 Callbacks

In [ ]:
# Reduce learning rate on plateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=2, verbose=1,
    mode='auto', min_delta=0.0001, cooldown=0, min_lr=1e-5
)

# save checkspoint
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="/gdrive/MyDrive/dataset/checkpoints/efficient_net",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# Early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.002, patience=3, verbose=1)

### 3.2 Fit

In [ ]:
# config
BATCH_SIZE = 256
EPOCH = 20
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
#input:
train, validation = get_data()

Found 105392 files belonging to 42 classes.
Using 94853 files for training.
Found 105392 files belonging to 42 classes.
Using 94853 files for training.


In [ ]:
net = get_model()

try: 
    net.load_weights("/gdrive/MyDrive/dataset/checkpoints/efficient_net")
    print("Loaded weight from last check points")

except Exception as e:
    print("Check point not found", e)

# Train
history = net.fit(
    train,
    validation_data = validation,
    epochs= EPOCH,
    callbacks=[reduce_lr, model_checkpoint_callback],
    workers=AUTO    
)

115269632/115263384 [==============================] - 1s 0us/step
Loaded weight from last check points
Epoch 1/20
2965/2965 [==============================] - 2431s 813ms/step - loss: 0.6916 - accuracy: 0.8067 - val_loss: 0.6214 - val_accuracy: 0.8262
Epoch 2/20
2965/2965 [==============================] - 2344s 790ms/step - loss: 0.6838 - accuracy: 0.8080 - val_loss: 0.6115 - val_accuracy: 0.8291
Epoch 3/20
2965/2965 [==============================] - 2342s 790ms/step - loss: 0.6785 - accuracy: 0.8103 - val_loss: 0.6062 - val_accuracy: 0.8308
Epoch 4/20
2965/2965 [==============================] - 2351s 793ms/step - loss: 0.6721 - accuracy: 0.8094 - val_loss: 0.5995 - val_accuracy: 0.8331
Epoch 5/20
1513/2965 [==============>...............] - ETA: 9:44 - loss: 0.6629 - accuracy: 0.8145